# PBupsModelmusc with Algorithmic Differentiation in Julia
3 additional parameters were added to Bing’s accumulator model.  
To analyze the strong bias induced by unilateral FOF inactivation


In [2]:
# import packages..
import ForwardDiff
using ForwardDiff
# using PyPlot
import Base.convert
import Optim
using Optim

# === Upgrading from ForwardDiff v0.1 to v0.2
# instead of ForwardDiff.GradientNumber and ForwardDiff.HessianNumber, 
# we will use ForwardDiff.Dual

convert(::Type{Float64}, x::ForwardDiff.Dual) = Float64(x.value)
function convert(::Array{Float64}, x::Array{ForwardDiff.Dual}) 
    y = zeros(size(x)); 
    for i in 1:prod(size(x)) 
        y[i] = convert(Float64, x[i]) 
    end
    return y
end

immutable NumericPair{X,Y} <: Number
  x::X
  y::Y
end
Base.isless(a::NumericPair, b::NumericPair) = (a.x<b.x) || (a.x==b.x && a.y<b.y)


isless (generic function with 46 methods)

To evaluate how well a particular set of parameter values $\theta$ fits the behavioral data, we compute the probability of oberving the data given the model.

For each trial $i$, we will compute the likelihood of seeing the data under the model assuming that trials are independent. 

$P(D|\theta) = \prod_{i}P(d_i|t_{i,R},t_{i,L},\theta)$

$t_{i,R},t_{i,L}$ : the right and left click times on trial $i$

$d_i$ : the subject's decision on trial $i$

The best-fit parameter values are the parameters $\theta$ that maximize the likelihood (Maximum likelihood values)

To help maximize the likelihood(or log likelihood), we will compute the derivative $\partial P(d_i|t_{i,R},t_{i,L},\theta) / \partial\theta$ for each of the parameters in the set $\theta$.

After we get these gradients of 9 model parameters, we will apply them for optimization.

## Import data 

In [43]:
using MAT
ratdata = matread("chrono_fof_rawdata.mat")
# ratdata = matread("testdata.mat")
# ratdata = matread("chrono_B069_rawdata.mat")

Dict{ASCIIString,Any} with 2 entries:
  "rawdata"      => Dict{ASCIIString,Any}("is_probe"=>1x3836 Array{Any,2}:…
  "orig_rawdata" => Dict{ASCIIString,Any}("is_probe"=>1x3836 Array{Any,2}:…

In [44]:
function trialdata(rawdata, trial::Int)
    if rawdata["pokedR"][trial] > 0
        rat_choice = 1;  # "R"
    else
        rat_choice = -1; # "L"
    end;
    
    if typeof(rawdata["rightbups"][trial]) <: Array
        rvec = vec(rawdata["rightbups"][trial])::Array{Float64,1};
    else
        rvec = Float64[]
    end
    if typeof(rawdata["leftbups"][trial]) <: Array
        lvec = vec(rawdata["leftbups"][trial])::Array{Float64,1};
    else
        lvec = Float64[]
    end
    
    return rvec, lvec, 
    rawdata["T"][trial]::Float64, rat_choice
end



trialdata (generic function with 1 method)

In [47]:
@time RightClickTimes, LeftClickTimes, maxT, rat_choice = trialdata(ratdata["rawdata"], 1)

  0.000037 seconds (15 allocations: 528 bytes)


([0.0,0.012634999999999999],[0.0,0.028775000000000002,0.046955,0.07099,0.10994,0.12017,0.21356,0.21445,0.230355,0.26368,0.266795],0.2796896469572883,-1)

## bin_centers = make_bins(B, dx, binN)

In [48]:
"""
function bin_centers = make_bins(B, dx, binN)

Makes a series of points that will indicate bin centers. The first and
last points will indicate sticky bins. No "bin edges" are made-- the edge
between two bins is always implicity at the halfway point between their
corresponding centers. The center bin is always at x=0; bin spacing
(except for last and first bins) is always dx; and the position
of the first and last bins is chosen so that |B| lies exactly at the
midpoint between 1st (sticky) and 2nd (first real) bins, as well as
exactly at the midpoint between last but one (last real) and last
(sticky) bins.

Playing nice with ForwardDiff means that the *number* of bins must be predetermined.
So this function will not actually set the number of bins; what it'll do is determine their
locations. To accomplish this separation, the function uses as a third parameter binN,
which should be equal to the number of bins with bin centers > 0, as follows: 
   binN = ceil(B/dx)
and then the total number of bins will be 2*binN+1, with the center one always corresponding
to position zero. Use non-differentiable types for B and dx for this to work.
"""

function make_bins{T}(bins::Vector{T}, B, dx::T, binN)
    cnt = 1
    for i=-binN:binN
        bins[cnt] = i*dx
        cnt = cnt+1
    end
    
    if binN*dx == B
        bins[end] = B + dx
        bins[1] = -B - dx
    else
        bins[end] = 2*B - (binN-1)*dx
        bins[1] = -2*B + (binN-1)*dx
    end
end;

In [49]:
binN = ceil(4.1/0.25)
bins = zeros(typeof(binN), Int(binN*2+1))
@time make_bins(bins,4.1,0.25,binN)
bins

  0.012024 seconds (5.57 k allocations: 263.271 KB)


35-element Array{Float64,1}:
 -4.2 
 -4.0 
 -3.75
 -3.5 
 -3.25
 -3.0 
 -2.75
 -2.5 
 -2.25
 -2.0 
 -1.75
 -1.5 
 -1.25
  ⋮   
  1.5 
  1.75
  2.0 
  2.25
  2.5 
  2.75
  3.0 
  3.25
  3.5 
  3.75
  4.0 
  4.2 

In [50]:
B = 4.1
dx_test = 0.25
binN = ceil(B/dx_test)
bins = zeros(typeof(binN), Int(binN*2+1))
@profile make_bins(bins,B,dx_test,binN)
Profile.print()
Profile.clear_malloc_data() 
bin_centers = bins
bins

3   ...ia/lib/julia/sys.dylib; trunc; (unknown line)
724 task.jl; anonymous; line: 447
 724 .../IJulia/src/IJulia.jl; eventloop; line: 143
  724 ...rc/execute_request.jl; execute_request_0x535c5df2; line: 183
   724 loading.jl; include_string; line: 282
    1   ...a/lib/julia/sys.dylib; typeinf_ext; (unknown line)
     1 ...a/lib/julia/sys.dylib; typeinf; (unknown line)
      1 ...a/lib/julia/sys.dylib; typeinf_uncached; (unknown line)
       1 ...a/lib/julia/sys.dylib; abstract_eval; (unknown line)
        1 .../lib/julia/sys.dylib; abstract_eval_call; (unknown line)
         1 .../lib/julia/sys.dylib; abstract_call; (unknown line)
          1 ...lib/julia/sys.dylib; abstract_call_gf; (unknown line)
           1 ...lib/julia/sys.dylib; typeinf; (unknown line)
            1 ...lib/julia/sys.dylib; typeinf; (unknown line)
             1 ...ib/julia/sys.dylib; typeinf_uncached; (unknown line)
              1 ...ib/julia/sys.dylib; abstract_interpret; (unknown line)
               1 ...b/

35-element Array{Float64,1}:
 -4.2 
 -4.0 
 -3.75
 -3.5 
 -3.25
 -3.0 
 -2.75
 -2.5 
 -2.25
 -2.0 
 -1.75
 -1.5 
 -1.25
  ⋮   
  1.5 
  1.75
  2.0 
  2.25
  2.5 
  2.75
  3.0 
  3.25
  3.5 
  3.75
  4.0 
  4.2 

## Global Variables

In [51]:
# Global variables 
const epsilon = 10.0^(-10);
const dx = 0.25;
const dt = 0.02;
const total_rate = 40;

## Parameters

a : decision variable, memory accumulator

$$ da =
  \begin{cases}
    0       & \quad \text{if, } |a| \geq B \\
    \sigma_adW + (\delta_{t,t_R} \cdot \eta C(t) - \delta_{t,t_L} \cdot \eta C(t))dt + \lambda adt  & \quad \text{otherwise, }\\
  \end{cases}
$$



The impact of each click (C) is affected by sensory adaptation that depends on clicks from both right and left sides:

$$ 
\frac{\mathrm d C}{\mathrm d t} = \frac{1-C}{\tau_\phi} + (1-\phi)C(\delta_{t,t_R}+\delta_{t,t_L}) 
$$


sigma2_a ($\sigma_a^2$) : a diffusion constant, parameterizing noise in a.

sigma2_s ($\sigma_s^2$) : parameterizing noise when adding evidence from a right or left pulse. (incoming sensory evidence)

sigma2_i ($\sigma_i^2$) : initial condition for the dynamical equation at $t=0$

lam ($\lambda$) : consistent drift in the memory a ($\lambda<0$ : leaky or forgetful case, $\lambda>0$ : unstable or impulsive case)

B : decision bound

bias : bias parameter determines the position of the threshold in a (which a Rightward decision is made)

phi ($\phi$) : parameterize sensory adaptation (by defining the dynamics of C ($\phi>1$ : Facilitation, $\phi<1$ : Depression, $\phi=1$ : absense of sensory adaptation)

tau_phi ($\tau_\phi$) :

lapse : The lapse rate parameterizes the probability of making a random response.

______
biased_sigma2_s :  
biased_input :  
biased_lapse :   


In [52]:
sigma_a = 1; sigma_s = 0.1; sigma_i = 0.2; 
sigma_a_sbin = sigma_a  # remember we need this copy for Fmatrix
lam = -0.0005; B = 4.1; bias = 0.1; 
phi = 0.3; tau_phi = 0.1; lapse = 0.05*2;
biased_sigma2_s = 0.1; biased_input = 0.1; biased_lapse = 0.1;
params = [sigma_a, sigma_s, sigma_i, lam, B, bias, phi, tau_phi, lapse, biased_sigma2_s, biased_input, biased_lapse]   

sigma = params[1];
lam   = params[2];
c     = params[3];

## F = Fmatrix([sigma, lambda, c], bin_centers)

In [53]:
"""
function F = Fmatrix([sigma, lambda, c], bin_centers)

Uses globals
    dt
    dx
    epsilon       (=10.0^-10)

Returns a square Markov matrix of transition probabilities. 
Plays nice with ForwardDiff-- that is why bin_centers is a global vector (so that the rem
operations that go into defining the bins, which ForwardDiff doesn't know how to deal with,
stay outside of this differentiable function)

sigma  should be in (accumulator units) per (second^(1/2))
lambda should be in s^-1
c      should be in accumulator units per second
bin_centers should be a vector of the centers of all the bins. Edges will be at midpoints
       between the centers, and the first and last bin will be sticky.

dx is not used inside Fmatrix, because bin_centers specifies all we need to know.
dt *is* used inside Fmatrix, to convert sigma, lambda, and c into timestep units
"""
function Fmatrix{T}(F::AbstractArray{T,2},params::Vector, bin_centers)
    sigma2 = params[1];
    lam   = params[2];
    c     = params[3];
    
    sigma2_sbin = convert(Float64, sigma2)
      
    n_sbins = max(70, ceil(10*sqrt(sigma2_sbin)/dx))
    
    swidth = 5*sqrt(sigma2_sbin)
    sbinsize = swidth/n_sbins;#sbins[2] - sbins[1]
    base_sbins    = collect(-swidth:sbinsize:swidth)
    
    ps       = exp(-base_sbins.^2/(2*sigma2))
    ps       = ps/sum(ps);
    
    sbin_length = length(base_sbins)
    binN = length(bin_centers)

    mu = 0.
    for j in 2:binN
        if lam == 0
            mu = bin_centers[j]*exp(lam*dt)
        else
            mu = (bin_centers[j] + c/lam)*exp(lam*dt) - c/lam
        end
        
        for k in 1:sbin_length
            sbin = (k-1)*sbinsize + mu - swidth
             
            if sbin < bin_centers[1] #(bin_centers[1] + bin_centers[2])/2
                F[1,j] = F[1,j] + ps[k]
            elseif bin_centers[end] <= sbin#(bin_centers[end]+bin_centers[end-1])/2 <= sbins[k]
                F[end,j] = F[end,j] + ps[k]
            else # more condition
                if (sbin > bin_centers[1] && sbin < bin_centers[2])
                    lp = 1; hp = 2;
                elseif (sbin > bin_centers[end-1] && sbin < bin_centers[end])
                    lp = binN-1; hp = binN;
                else 
                    lp = floor(Int,((sbin-bin_centers[2])/dx) + 2)#find(bin_centers .<= sbins[k])[end]
                    hp = lp+1#Int(ceil((sbins[k]-bin_centers[2])/dx) + 1);
                end

                if lp < 1 
                    lp = 1; 
                end
                if hp < 1 
                    hp = 1;
                end

                if lp == hp
                    F[lp,j] = F[lp,j] + ps[k]
                else
                    dd = bin_centers[hp] - bin_centers[lp]
                    F[hp,j] = F[hp,j] + ps[k]*(sbin - bin_centers[lp])/dd
                    F[lp,j] = F[lp,j] + ps[k]*(bin_centers[hp] - sbin)/dd
                end                   
            end
        end
    end
    F[:,1] = 0; F[:,end] = 0; F[1,1] = 1; F[end,end] = 1;
end

Fmatrix (generic function with 1 method)

In [54]:
F = zeros(typeof(0.2),length(bin_centers),length(bin_centers))
@time Fmatrix(F,[0.2, 0., 0.0],bin_centers) # Fi
F

  0.078094 seconds (36.20 k allocations: 1.603 MB)


35x35 Array{Float64,2}:
 1.0  0.41182      0.218907     0.0917964    …  0.0          0.0          0.0
 0.0  0.197142     0.172131     0.112014        0.0          0.0          0.0
 0.0  0.187228     0.217923     0.187228        0.0          0.0          0.0
 0.0  0.119939     0.187228     0.217923        0.0          0.0          0.0
 0.0  0.0571377    0.119939     0.187228        0.0          0.0          0.0
 0.0  0.0202246    0.0571377    0.119939     …  0.0          0.0          0.0
 0.0  0.00531176   0.0202246    0.0571377       0.0          0.0          0.0
 0.0  0.00103206   0.00531176   0.0202246       0.0          0.0          0.0
 0.0  0.000148294  0.00103206   0.00531176      0.0          0.0          0.0
 0.0  1.59548e-5   0.000148294  0.00103206      0.0          0.0          0.0
 0.0  1.11105e-6   1.59548e-5   0.000148294  …  0.0          0.0          0.0
 0.0  0.0          1.11105e-6   1.59548e-5      0.0          0.0          0.0
 0.0  0.0          0.0          1.11105e

In [55]:
F = zeros(typeof(params[1]),length(bin_centers),length(bin_centers))
@time Fmatrix(F,params,bin_centers)
F

  0.000109 seconds (13 allocations: 7.547 KB)


35x35 Array{Float64,2}:
 1.0  0.461125    0.365007    0.274946    …  0.0         0.0         0.0
 0.0  0.0908452   0.0859592   0.0811359      0.0         0.0         0.0
 0.0  0.0945948   0.100835    0.0946575      0.0         0.0         0.0
 0.0  0.0888839   0.0945533   0.100892       0.0         0.0         0.0
 0.0  0.0737024   0.0889555   0.0945117      0.0         0.0         0.0
 0.0  0.0612142   0.0736883   0.0890271   …  0.0         0.0         0.0
 0.0  0.044833    0.0612781   0.0736743      0.0         0.0         0.0
 0.0  0.0329193   0.0448354   0.0613421      0.0         0.0         0.0
 0.0  0.0212918   0.0329616   0.0448377      0.0         0.0         0.0
 0.0  0.0138235   0.021298    0.0330038      0.0         0.0         0.0
 0.0  0.0078944   0.0138445   0.0213041   …  0.0         0.0         0.0
 0.0  0.00453257  0.0078985   0.0138655      0.0         0.0         0.0
 0.0  0.00228512  0.00454054  0.00790261     3.13941e-7  0.0         0.0
 ⋮                         

## logProbRight 
### (params::Vector, RightClickTimes::Vector, LeftClickTimes::Vector, Nsteps::Int)

* params = [sigma_a, sigma_s, sigma_i, lambda, B, bias, phi, tau_phi, lapse]
* RightClickTimes vector with elements indicating times of right clicks
* LeftClickTimes vector with elements indicating times of left clicks
* Nsteps number of timesteps to simulate 

a (column vector representing distribution of values of accumulator a)

a_trace (length(bin_centers)-by-Nsteps+1), a trace of the distribution of a as 
    a function of time
    
c_trace (row vector Nsteps+1 long, effective value of c as 
    a function of time after adaptation)


In [56]:
"""
version with inter-click interval(ici) for c_eff_net / c_eff_tot (followed the matlab code) 
(which was using dt for c_eff)

function logProbRight(params::Vector, RightClickTimes::Vector, LeftClickTimes::Vector, Nsteps::Int)

    Nsteps            number of timesteps to simulate
    RightClickTimes   vector with elements indicating times of right clicks
    LeftClickTimes    vector with elements indicating times of left clicks

    a      (column vector representing distribution of values of accumulator a)

    a_trace (length(bin_centers)-by-Nsteps+1), a trace of the distribution of a as 
            a function of time
    c_trace (row vector Nsteps+1 long, effective value of c as 
            a function of time after adaptation)

Takes params
    sigma_a = params[1]; sigma_s = params[2]; sigma_i = params[3]; 
    lambda = params[4]; B = params[5]; bias = params[6]; 
    phi = params[7]; tau_phi = params[8]; lapse = params[9]

Returns the log of the probability that the agent chose Right. 
"""

function logProbRight(params::Vector, RightClickTimes::Array{Float64,1}, LeftClickTimes::Array{Float64,1}, Nsteps::Int)
    sigma_a = params[1]; sigma_s = params[2]; sigma_i = params[3]; 
    lambda = params[4]; B = params[5]; bias = params[6]; 
    phi = params[7]; tau_phi = params[8]; lapse = params[9]
    biased_sigma2_s = params[10]; biased_input = params[11]; biased_lapse = params[12];
    
    if isempty(RightClickTimes) RightClickTimes = zeros(0) end;
    if isempty(LeftClickTimes ) LeftClickTimes  = zeros(0) end;
    
    NClicks = zeros(Int, Nsteps); 
    Lhere  = zeros(Int, length(LeftClickTimes)); 
    Rhere = zeros(Int, length(RightClickTimes)); 
    
    for i in 1:length(LeftClickTimes)
        Lhere[i] = ceil((LeftClickTimes[i]+epsilon)/dt)
    end
    for i in 1:length(RightClickTimes)
        Rhere[i] = ceil((RightClickTimes[i]+epsilon)/dt)
    end    
    
    for i in Lhere  
        NClicks[Int(i)] = NClicks[Int(i)]  + 1
    end
    for i in Rhere  
        NClicks[Int(i)] = NClicks[Int(i)]  + 1
    end
    
    # === Upgrading from ForwardDiff v0.1 to v0.2
    # instead of using convert we can use floor(Int, ForwardDiff.Dual) and
    # ceil(Int, ForwardDiff.Dual)

    binN = ceil(Int, B/dx)#Int(ceil(my_B/dx))  
    binBias = floor(Int, bias/dx) + binN+1  
    bin_centers = zeros(typeof(dx), binN*2+1)
    make_bins(bin_centers, B, dx, binN)  

    ## biased params added
    a0 = zeros(typeof(sigma_a),length(bin_centers))
    a0[binN+1] = 1-lapse/2-biased_lapse/2; a0[1] = biased_lapse/2; a0[end] = lapse/2;
    
    temp_l = [NumericPair(LeftClickTimes[i],-1) for i=1:length(LeftClickTimes)]
    temp_r = [NumericPair(RightClickTimes[i],1) for i=1:length(RightClickTimes)]
    allbups = sort!([temp_l; temp_r])

    c_eff = 0.
    cnt = 0
        
    Fi = zeros(typeof(sigma_i),length(bin_centers),length(bin_centers))
    Fmatrix(Fi,[sigma_i, 0., 0.0], bin_centers)
    a = Fi*a0;
    
    F0 = zeros(typeof(sigma_a),length(bin_centers),length(bin_centers))
    Fmatrix(F0,[sigma_a*dt, lambda, 0.0], bin_centers)
    for i in 2:Nsteps 
        c_eff_tot = 0.
        c_eff_net = 0.
        net_c_input = 0.
        net_i_input = 0.
        
        if NClicks[i-1]==0
            c_eff_tot = 0.
            c_eff_net = 0.
            a = F0*a
        else
            for j in 1:NClicks[i-1]
                if cnt != 0 || j != 1
                    ici = allbups[cnt+j].x - allbups[cnt+j-1].x
                    c_eff = 1 + (c_eff*phi - 1)*exp(-ici/tau_phi)
                    c_eff_tot = c_eff_tot + c_eff
                    c_eff_net = c_eff_net + c_eff*allbups[cnt+j].y
                    
                    ## biased params added
                    net_c_input = (c_eff_tot+c_eff_net)/2
                    net_i_input = c_eff_tot-c_eff_net
                    
                    c_eff_net = net_c_input*biased_input - net_i_input
                end
                if j == NClicks[i-1]
                    cnt = cnt+j
                end
            end
            
            gain_first = 0
            ## biased params added
            if gain_first == 1
                net_sigma = sigma_a*dt + (sigma_s*net_i_input)/total_rate + net_c_input*biased_input*biased_sigma2_s/total_rate
            else
                net_sigma = sigma_a*dt + (sigma_s*net_i_input)/total_rate + net_c_input*biased_sigma2_s/total_rate
            end
            F = zeros(typeof(net_sigma),length(bin_centers),length(bin_centers))
            Fmatrix(F,[net_sigma, lambda, c_eff_net/dt], bin_centers)
            a = F*a
        end
    end
#     plot(1:Nsteps+1,c_trace[:])    
#     imshow(a_trace, interpolation="none")
    pright = sum(a[binBias+2:end]) + 
    a[binBias]*((bin_centers[binBias+1] - bias)/dx/2) +
    a[binBias+1]*(0.5 + (bin_centers[binBias+1] - bias)/dx/2)
    
    return log(pright)
end

function logLike(params::Vector, RightClickTimes::Array{Float64,1}, LeftClickTimes::Array{Float64,1}, Nsteps::Int, rat_choice::Int)
    if rat_choice > 0
        # println("Right")
        return logProbRight(params, RightClickTimes, LeftClickTimes, Nsteps)
    elseif rat_choice < 0
        # println("Left")
        return log(1 - exp(logProbRight(params, RightClickTimes, LeftClickTimes, Nsteps)))
    end
end

logLike (generic function with 1 method)

## single_trial
### (params::Vector, RightClickTimes::Vector, LeftClickTimes::Vector, Nsteps::Int, rat_choice::Int)

In [57]:
""" 
function (LL, LLgrad) = 
    single_trial(params::Vector, RightClickTimes::Vector, LeftClickTimes::Vector, Nsteps::Int, rat_choice::Int)

Computes the log likelihood according to Bing's model, and returns log likelihood, gradient

params is a vector whose elements, in order, are
    sigma_a    square root of accumulator variance per unit time sqrt(click units^2 per second)
    sigma_s    standard deviation introduced with each click (will get scaled by click adaptation)
    sigma_i    square root of initial accumulator variance sqrt(click units^2)
    lambda     1/accumulator time constant (sec^-1). Positive means unstable, neg means stable
    B          sticky bound height (click units)
    bias       where the decision boundary lies (click units)
    phi        click adaptation/facilitation multiplication parameter
    tau_phi    time constant for recovery from click adaptation (sec)
    lapse      2*lapse fraction of trials are decided randomly

rat_choice     should be either "R" or "L"


RETURNS:


"""
# === Upgrading from ForwardDiff v0.1 to v0.2
# for Retrieving Lower-Order Results
#     # old way
#     answer, results = ForwardDiff.hessian(f, x, AllResults)
#     v = ForwardDiff.value(results)
#     g = ForwardDiff.gradient(results)
#     h = ForwardDiff.hessian(results) # == answer

#     # new way
#     out = HessianResult(x)
#     ForwardDiff.hessian!(out, f, x)
#     v = ForwardDiff.value(out)
#     g = ForwardDiff.gradient(out)
#     h = ForwardDiff.hessian(out)

function single_trial(params::Vector, RightClickTimes::Array{Float64,1}, LeftClickTimes::Array{Float64,1}, Nsteps::Int, rat_choice::Int)
    function llikey(params::Vector)
        logLike(params, RightClickTimes, LeftClickTimes, Nsteps, rat_choice)
    end

    result =  GradientResult(params)
    
    ForwardDiff.gradient!(result, llikey, params);
    
    LL     = ForwardDiff.value(result)
    LLgrad = ForwardDiff.gradient(result)
   
    return LL, LLgrad
end

# function single_trial(params::Vector, RightClickTimes::Vector, LeftClickTimes::Vector, Nsteps::Int, rat_choice::Int)
#     function llikey(params::Vector)
#         logLike(params, RightClickTimes, LeftClickTimes, Nsteps, rat_choice)
#     end

#     result =  HessianResult(params) 
#     ForwardDiff.hessian!(result, llikey, params);
    
#     LL     = ForwardDiff.value(result)
#     LLgrad = ForwardDiff.gradient(result)
#     LLhessian = ForwardDiff.hessian(result)
   
#     return LL, LLgrad, LLhessian
# end

single_trial (generic function with 1 method)

In [58]:
### =============== testing 1 ================= ####

# Parameters
sigma_a = 1; sigma_s = 0.1; sigma_i = 0.2; 
sigma_a_sbin = sigma_a  # remember we need this copy for Fmatrix
lam = -0.0005; B = 4.1; bias = 0.1; 
phi = 0.3; tau_phi = 0.1; lapse = 0.05*2;
biased_sigma2_s = 0.1; biased_input = 0.1; biased_lapse = 0.1;
params = [sigma_a, sigma_s, sigma_i, lam, B, bias, phi, tau_phi, lapse, biased_sigma2_s, biased_input, biased_lapse]   

RightClickTimes, LeftClickTimes, maxT, rat_choice = trialdata(ratdata["rawdata"], 1)
Nsteps = Int(cld(maxT,dt))

@time logLike(params, RightClickTimes, LeftClickTimes, Nsteps, rat_choice)
### =========================================== #### 

  0.205939 seconds (126.77 k allocations: 6.958 MB)


-0.05129329440509444

In [59]:
### =============== testing 2 ================= ####
@time LL, LLgrad = single_trial(params, RightClickTimes, LeftClickTimes, Nsteps, rat_choice)
println(LL)
println(LLgrad)
### =========================================== ####

  0.333616 seconds (326.80 k allocations: 16.594 MB, 4.25% gc time)
-0.05129329440509444
[-1.6942882237111069e-10,-8.369562599103428e-11,-5.855995501544715e-10,-4.485361648602246e-11,-0.0,9.746881317511218e-11,7.074143847252285e-10,-5.664529847780174e-9,-0.5263157894731709,3.1970619189311883e-12,3.175728714281305e-11,9.74662462602276e-12]


In [60]:
LL

-0.05129329440509444

In [61]:
LLgrad

12-element Array{Float64,1}:
 -1.69429e-10
 -8.36956e-11
 -5.856e-10  
 -4.48536e-11
 -0.0        
  9.74688e-11
  7.07414e-10
 -5.66453e-9 
 -0.526316   
  3.19706e-12
  3.17573e-11
  9.74662e-12

# Maximize LL over parameter space
### Optimization with Optim.jl


In [62]:
function ComputeLL(params::Vector, ratdata, ntrials::Int)
    LL        = 0.
        
    for i in 1:ntrials
        RightClickTimes, LeftClickTimes, maxT, rat_choice = trialdata(ratdata, i)
        Nsteps = Int(ceil(maxT/dt))

        LLi = logLike(params, RightClickTimes, LeftClickTimes, Nsteps, rat_choice)
        LL        = LL + LLi;
    end
    
    LL = -LL
    return LL
end

ComputeLL (generic function with 1 method)

In [63]:
function ComputeGrad(params::Vector, ratdata, ntrials::Int)
    function WrapperLL(params::Vector)
        LL        = 0.

        for i in 1:ntrials
            RightClickTimes, LeftClickTimes, maxT, rat_choice = trialdata(ratdata, i)
            Nsteps = Int(ceil(maxT/dt))

            LLi = logLike(params, RightClickTimes, LeftClickTimes, Nsteps, rat_choice)
            LL        = LL + LLi;
        end

        LL = -LL
        return LL
    end

    result =  GradientResult(params)
    
    ForwardDiff.gradient!(result, WrapperLL, params);
    
    LL     = ForwardDiff.value(result)
    LLgrad = ForwardDiff.gradient(result)
   
    return LL, LLgrad
end

ComputeGrad (generic function with 1 method)

In [64]:
@profile ComputeLL(params,ratdata["rawdata"],300)
Profile.print()
Profile.clear_malloc_data() 

5    ...a/lib/julia/sys.dylib; trunc; (unknown line)
1147 task.jl; anonymous; line: 447
 1147 .../IJulia/src/IJulia.jl; eventloop; line: 143
  1147 ...rc/execute_request.jl; execute_request_0x535c5df2; line: 183
   1147 loading.jl; include_string; line: 282
    1    .../lib/julia/sys.dylib; typeinf_ext; (unknown line)
     1 ...a/lib/julia/sys.dylib; typeinf; (unknown line)
      1 ...a/lib/julia/sys.dylib; typeinf_uncached; (unknown line)
       1 ...a/lib/julia/sys.dylib; abstract_eval; (unknown line)
        1 .../lib/julia/sys.dylib; abstract_eval_call; (unknown line)
         1 .../lib/julia/sys.dylib; abstract_call; (unknown line)
          1 ...lib/julia/sys.dylib; abstract_call_gf; (unknown line)
           1 ...lib/julia/sys.dylib; typeinf; (unknown line)
            1 ...lib/julia/sys.dylib; typeinf; (unknown line)
             1 ...ib/julia/sys.dylib; typeinf_uncached; (unknown line)
              1 ...ib/julia/sys.dylib; abstract_interpret; (unknown line)
               1 .

In [66]:
@time ComputeGrad(params,ratdata["rawdata"],300)

 22.845179 seconds (6.12 M allocations: 8.060 GB, 4.58% gc time)


(2215.7268216003954,[-61.31240712313557,-7.516510891459202,-169.79971387805705,-14.824529052059342,-0.0,152.2663348435403,256.043113230891,-1938.052978139069,-4745.28135034643,-11.305349674594131,-762.1461496001752,39.05719226443167])

In [68]:
function ComputeHess(params::Vector, ratdata, ntrials::Int)
    function WrapperLL(params::Vector)
        LL        = 0.

        for i in 1:ntrials
            RightClickTimes, LeftClickTimes, maxT, rat_choice = trialdata(ratdata["rawdata"], i)
            Nsteps = Int(ceil(maxT/dt))

            LLi = logLike(params, RightClickTimes, LeftClickTimes, Nsteps, rat_choice)
            LL        = LL + LLi;
        end

        LL = -LL
        return LL
    end

    result =  HessianResult(params)
    
    ForwardDiff.hessian!(result, WrapperLL, params);
    
    LL     = ForwardDiff.value(result)
    LLgrad = ForwardDiff.gradient(result)
    LLHess = ForwardDiff.hessian(result)
   
    return LL, LLgrad, LLHess
end

ComputeHess (generic function with 1 method)

In [70]:
@time ComputeHess(params,ratdata,300)

166.292059 seconds (1.28 M allocations: 10.914 GB, 0.74% gc time)


(239.0649728461485,[-6.787142242484801,-0.8455089178893049,-18.446026275445494,-1.3078428165173557,-0.0,20.4510105856907,27.13017336870174,-212.36235760297487,-548.5504051326026,-1.223629648400699,-81.01237426141982,5.8905694475825605],
12x12 Array{Float64,2}:
   0.489222   -0.0584618   …   0.103961     -16.4752     1.82556 
  -0.0584618  -0.0203065      -0.00552267    -2.51362    0.265777
   3.23257     0.131166        0.681015     -20.1392     3.02202 
  -0.105241   -0.0282026       0.00125459    -5.37078    0.370996
  -0.0        -0.0            -0.0           -0.0       -0.0     
   1.82241     0.523783    …   0.286255      35.4516     2.17978 
  10.3433      0.947585        0.531215      29.7044    -7.10925 
 -80.4058     -6.90247        -4.24053     -176.104     54.6028  
  52.6406      7.14702         9.19968      552.679    -32.6535  
   0.103961   -0.00552267      0.0212137     -3.17445    0.325206
 -16.4752     -2.51362     …  -3.17445     -191.193     11.592   
   1.82556   

## Parallel Computing

In [ ]:
n_core = 4
if nworkers() < n_core
    addprocs(n_core-nworkers(); exeflags="--check-bounds=yes")
end
@assert nprocs() > n_core
@assert nworkers() >= n_core



In [ ]:
a = SharedArray(Float64,10)
@sync @parallel for i=1:10
  a[i] = i
end
a

In [ ]:
# addprocs(3)

# a = SharedArray(Float64,10)
# @parallel for i=1:10
#   a[i] = i
# end

function ComputeLL_par(LLs::SharedArray{Float64,1}, params::Vector, ratdata, ntrials::Int)
    @sync @parallel for i in 1:ntrials
        RightClickTimes, LeftClickTimes, maxT, rat_choice = trialdata(ratdata, i)
        Nsteps = Int(ceil(maxT/dt))
        LLs[i] = logLike(params, RightClickTimes, LeftClickTimes, Nsteps, rat_choice)
    end

    LL = sum(LLs)
    
    LL = -LL
    return LL
end

function ComputeGrad_par(LLs::SharedArray{Float64,1}, params::Vector, ratdata, ntrials::Int)
    function WrapperLL(params::Vector)
        LL        = 0.

        @sync @parallel for i in 1:ntrials
            RightClickTimes, LeftClickTimes, maxT, rat_choice = trialdata(ratdata, i)
            Nsteps = Int(ceil(maxT/dt))

            LLs[i] = logLike(params, RightClickTimes, LeftClickTimes, Nsteps, rat_choice)
        end
        
        LL = sum(LLs)
        LL = -LL
        return LL
    end

    result =  GradientResult(params)
    
    ForwardDiff.gradient!(result, WrapperLL, params);
    
    LL     = ForwardDiff.value(result)
    LLgrad = ForwardDiff.gradient(result)
   
    return LL, LLgrad
end


In [ ]:
LLs = SharedArray(Float64, 10)
@time SumLikey_LL_par(LLs,params, ratdata, 10)

In [ ]:
@profile SumLikey_LL_par(params, ratdata, 27) # sum of LL for trial (1-27)
Profile.print()
Profile.clear_malloc_data() 

In [ ]:
@time SumLikey_par(params, ratdata, 27)

In [ ]:
function Likely_all_trials{T}(LL::AbstractArray{T,1},params::Vector, ratdata, ntrials::Int)     
    for i in 1:ntrials
        RightClickTimes, LeftClickTimes, maxT, rat_choice = trialdata(ratdata, i)
        Nsteps = Int(ceil(maxT/dt))

        LL[i] = logLike(params, RightClickTimes, LeftClickTimes, Nsteps, rat_choice)
    end
end

In [ ]:
likely_all = zeros(ntrials)
@time Likely_all_trials(likely_all, params, ratdata["rawdata"], 27)
likely_all

In [89]:
# Parameters
sigma_a = 1; sigma_s = 0.1; sigma_i = 0.2; 
sigma_a_sbin = sigma_a  # remember we need this copy for Fmatrix
lam = -0.0005; B = 4.1; bias = 0.1; 
phi = 0.3; tau_phi = 0.1; lapse = 0.05*2;
biased_sigma2_s = 0.1; biased_input = 0.1; biased_lapse = 0.1;
params = [sigma_a, sigma_s, sigma_i, lam, B, bias, phi, tau_phi, lapse, biased_sigma2_s, biased_input, biased_lapse]   

ntrials = 3836

function LL_f(params::Vector)
    return ComputeLL(params, ratdata["rawdata"], ntrials)
end

function LL_g!(params::Vector, grads::Vector)
#     LL, LLgrad, LLhess = llikey(params)
    LL, LLgrad = ComputeGrad(params, ratdata["rawdata"], ntrials)
    for i=1:length(params)
        grads[i] = LLgrad[i]
    end
end

function LL_fg!(params::Vector, grads)
    LL, LLgrad = ComputeGrad(params, ratdata["rawdata"], ntrials)
    for i=1:length(params)
        grads[i] = LLgrad[i]
    end
    return LL
end

d4 = DifferentiableFunction(LL_f,
                            LL_g!,
                            LL_fg!)

Optim.DifferentiableFunction(LL_f,LL_g!,LL_fg!)

In [ ]:
# Parameters
sigma_a = 1; sigma_s = 0.1; sigma_i = 0.2; 
sigma_a_sbin = sigma_a  # remember we need this copy for Fmatrix
lam = -0.0005; B = 6.1; bias = 0.1; 
phi = 0.3; tau_phi = 0.1; lapse = 0.05*2;
biased_sigma2_s = 0.1; biased_input = 0.1; biased_lapse = 0.1;
# params = [sigma_a, sigma_s, sigma_i, lam, B, bias, phi, tau_phi, lapse, biased_sigma2_s, biased_input, biased_lapse]   
params = [2.63352, 0.287501, 0.501707, 1.91113, 6.23087, 0.487071, 0.537924, 0.69, 0.140861, 0.315909, 1.92412, 0.99205]

l = [0, 0, 0, -5, 5, -5, 0.01, 0.005, 0., 0., -1., 0.]
u = [200, 200, 30, +5, 25, +5, 1.2, 0.7, 1.,10000., 2., 1.3]

res = optimize(d4, params, l, u, Fminbox(); 
         optimizer = GradientDescent, optimizer_o = OptimizationOptions(g_tol = 1e-12,
                                                                        x_tol = 1e-10,
                                                                        f_tol = 1e-6,                                                                        
                                                                        iterations = 200,
                                                                        store_trace = true,
                                                                        show_trace = true,
                                                                        extended_trace = true))


Iter     Function value   Gradient norm 
 

In [84]:
x_bf = res.f_minimum

1538.5902305652457

In [85]:
res

Results of Optimization Algorithm
 * Algorithm: Fminbox with Gradient Descent
 * Starting Point: [1.0,0.1, ...]
 * Minimizer: [2.6335164932543877,0.2875014302039217, ...]
 * Minimum: 1.538590e+03
 * Iterations: 114
 * Convergence: true
   * |x - x'| < 1.0e-10: true
   * |f(x) - f(x')| / |f(x)| < 1.0e-06: true
   * |g(x)| < 1.0e-12: false
   * Reached Maximum Number of Iterations: false
 * Objective Function Calls: 1581
 * Gradient Calls: 1581

In [86]:
res.minimum

12-element Array{Float64,1}:
 2.63352 
 0.287501
 0.501707
 1.91113 
 6.23087 
 0.487071
 0.537924
 0.7     
 0.140861
 0.315909
 1.92412 
 0.992052

In [88]:
@time LL, LLgrad, LLhess = ComputeHess(params,ratdata,ntrials)

2716.705393 seconds (15.69 M allocations: 195.042 GB, 1.03% gc time)


(2215.7269936287066,[-61.311538211652774,-7.516509432373704,-169.79727192940504,-14.823724771025638,-0.0,152.26638046963237,256.04336675541975,-1938.0555785679196,-4745.283329985102,-11.305197126892882,-762.1438015465878,39.05709726491926],
12x12 Array{Float64,2}:
   10.1809       0.0817604  …    2.16681      -85.8296    13.7254 
    0.0817604   -0.111279        0.0659889    -16.0026     2.13824
   37.5209       2.27336         7.55192      -65.9949    18.2869 
    1.54624      0.0828142       0.725397     -22.1618     2.76159
   -0.0         -0.0            -0.0           -0.0       -0.0    
   10.2121       3.66181    …    0.985407     101.45      75.3805 
   78.2281       6.20482         2.13142      153.591    -59.2337 
 -569.29       -44.5143         -9.83036     -451.038    433.87   
  475.832       63.6896         84.1062      5193.32    -328.536  
    2.16681      0.0659889       0.479033     -14.8242     2.52523
  -85.8296     -16.0026     …  -14.8242     -1211.42      42.6067

In [ ]:
function main()
    
    ratname = "fof"
    # data import
    ratdata = matread(*("chrono_",ratname,"_rawdata.mat"))
    println("rawdata of ", ratname, " imported" )

    # number of trials
    ntrials = Int(ratdata["total_trials"])

    # Parameters
    sigma_a = 1.; sigma_s = 0.1; sigma_i = 0.2; 
    sigma_a_sbin = sigma_a  # remember we need this copy for Fmatrix
    lam = -0.0005; B = 6.1; bias = 0.1; 
    phi = 0.3; tau_phi = 0.1; lapse = 0.05*2;
    biased_sigma2_s = 0.1; biased_input = 0.1; biased_lapse = 0.1;
    params = [sigma_a, sigma_s, sigma_i, lam, B, bias, phi, tau_phi, lapse, biased_sigma2_s, biased_input, biased_lapse]   

    l = [0, 0, 0, -5, 5, -5, 0.01, 0.005, 0., 0., -1., 0.]
    u = [200, 200, 30, +5, 25, +5, 1.2, 0.7, 1.,10000., 2., 1.3]

    # @code_warntype SumLikey(params, ratdata, ntrials)

    function LL_f(params::Vector)
        return SumLikey_LL(params, ratdata, ntrials)
    end

    function LL_g!(params::Vector, grads::Vector)
        LL, LLgrad = SumLikey(params, ratdata, ntrials)
        for i=1:length(params)
            grads[i] = LLgrad[i]
        end
    end

    function LL_fg!(params::Vector, grads)
        LL, LLgrad = SumLikey(params, ratdata, ntrials)
        for i=1:length(params)
            grads[i] = LLgrad[i]
        end
        return LL
    end

    d4 = DifferentiableFunction(LL_f,
                                LL_g!,
                                LL_fg!)

    res = optimize(d4, params, l, u, Fminbox(); 
             optimizer = GradientDescent, optimizer_o = OptimizationOptions(g_tol = 1e-12,
                                                                            iterations = 200,
                                                                            store_trace = true,
                                                                            show_trace = true))
    LL, LLgrad, LLhess = ComputeHess(params,ratdata,ntrials)
    
end

In [ ]:
main()

In [ ]:
# make grid of params. calculate likelihood -> make likelihood landscape